In [1]:
#| default_exp cli

In [2]:
#| export
from __future__ import annotations

# cli

> Command-line interface for nooprompter

<!-- # Prologue -->


In [3]:
#| export
from pathlib import Path

from fastcore.script import call_parse


In [4]:
#| export
from nooprompter.config import find_config
from nooprompter.config import load_config
from nooprompter.config import validate_config
from nooprompter.tags import discover_tags_in_templates
from nooprompter.templates import process_all
from nooprompter.templates import process_vendor


In [5]:
#| hide
import os
import tempfile

from fastcore.test import *


# CLI Commands

## init

In [6]:
#| export
@call_parse
def init(
    force: bool = False  # Overwrite existing config
):
    "Initialize nooprompter config from default template"
    config_file = Path('.nooprompter.yml')
    
    if config_file.exists() and not force:
        print(f"Config already exists: {config_file}")
        print("Use --force to overwrite")
        return 1
    
    try:
        from importlib.resources import files
        default_yml = files('nooprompter').joinpath('static/default.nooprompter.yml')
        with default_yml.open('r') as src:
            config_file.write_text(src.read())
        
        print(f"✓ Created {config_file}")
        print("\nNext steps:")
        print("  1. Edit .nooprompter.yml to customize tags")
        print("  2. Create templates/ directory structure")
        print("  3. Run 'nooprompter build' to process templates")
        return 0
    except Exception as e:
        print(f"Error: Could not create config file: {e}")
        return 1

In [7]:
with tempfile.TemporaryDirectory() as tmpdir:
    old_cwd = os.getcwd()
    try:
        os.chdir(tmpdir)
        
        # Test 1: Fresh init
        result = init()
        test_eq(result, 0)
        test_eq(Path('.nooprompter.yml').exists(), True)
        
        # Test 2: Already exists without force
        result = init()
        test_eq(result, 1)
        
        # Test 3: Force overwrite
        result = init(force=True)
        test_eq(result, 0)
    finally:
        os.chdir(old_cwd)

✓ Created .nooprompter.yml

Next steps:
  1. Edit .nooprompter.yml to customize tags
  2. Create templates/ directory structure
  3. Run 'nooprompter build' to process templates
Config already exists: .nooprompter.yml
Use --force to overwrite
✓ Created .nooprompter.yml

Next steps:
  1. Edit .nooprompter.yml to customize tags
  2. Create templates/ directory structure
  3. Run 'nooprompter build' to process templates


## config

In [8]:
#| export
@call_parse
def config_cmd(
    validate: bool = False  # Validate configuration
):
    "Show and optionally validate configuration"
    cfg = load_config()
    config_path = find_config()
    
    # Show config info
    print(f"Config: {config_path if config_path else '(using defaults)'}")
    print(f"Tags: {len(cfg.get('tags', {}))}")
    
    # List template vendors
    templates = cfg.get('templates', {})
    vendor_count = len([k for k in templates.keys() if k != 'meta'])
    print(f"Vendors: {vendor_count}")
    for key in templates.keys():
        if key == 'meta':
            print(f"  • meta (project files)")
        else:
            print(f"  • {key}")
    
    # Validate if requested
    if validate:
        is_valid, errors = validate_config(cfg)
        if is_valid:
            print("\n✓ Configuration is valid")
        else:
            print("\n✗ Configuration errors:")
            for error in errors:
                print(f"  - {error}")
            return 1
    
    return 0

In [9]:
# Test with real config
result = config_cmd()
test_eq(result, 0)

result = config_cmd(validate=True)
test_eq(result, 0)

# Test with defaults (no config file)
with tempfile.TemporaryDirectory() as tmpdir:
    old_cwd = os.getcwd()
    try:
        os.chdir(tmpdir)
        result = config_cmd()
        test_eq(result, 0)
    finally:
        os.chdir(old_cwd)

Config: /Users/vic/dev/repo/project/nooprompter/.nooprompter.yml
Tags: 7
Vendors: 2
  • meta (project files)
  • Cursor
  • Claude_Coder
Config: /Users/vic/dev/repo/project/nooprompter/.nooprompter.yml
Tags: 7
Vendors: 2
  • meta (project files)
  • Cursor
  • Claude_Coder

✓ Configuration is valid
Config: (using defaults)
Tags: 7
Vendors: 2
  • meta (project files)
  • Cursor
  • Claude_Coder


## tags

In [10]:
#| export
@call_parse
def tags(
    path: str|None = None  # Path to template file or directory
):
    "Discover tags in templates"
    config = load_config()
    
    # Find base path from config location
    config_path = find_config()
    base_path = config_path.parent if config_path else Path.cwd()
    
    if path:
        # Scan specific path (absolute or relative to cwd)
        target = Path(path)
        if not target.exists():
            print(f"Error: Path not found: {path}")
            return 1
        
        if target.is_file():
            from nooprompter.tags import discover_tags_in_file
            found_tags = discover_tags_in_file(target)
            print(f"Tags in {path}:")
        else:
            found = discover_tags_in_templates(target)
            found_tags = set()
            for file_tags in found.values():
                found_tags.update(file_tags)
            print(f"Tags in {path}:")
    else:
        # Scan templates directory relative to config location
        templates_dir = base_path / 'templates'
        if not templates_dir.exists():
            print(f"Templates directory not found at {templates_dir}")
            print("Create 'templates/' or specify path with --path")
            return 1
        
        found = discover_tags_in_templates(templates_dir)
        found_tags = set()
        for file_tags in found.values():
            found_tags.update(file_tags)
        print(f"Tags in {templates_dir.relative_to(base_path)}:")
    
    if not found_tags:
        print("  (no tags found)")
        return 0
    
    # Show tags with status
    defined_tags = config.get('tags', {})
    for tag in sorted(found_tags):
        if tag in defined_tags:
            print(f"  <{tag}> ✓")
        else:
            print(f"  <{tag}> ✗ (not defined)")
    
    return 0

In [11]:
# Test with real templates
result = tags()
test_eq(result, 0)

# Test with specific file
template_file = Path('../templates/Cursor/project.mdc')
if template_file.exists():
    result = tags(path=str(template_file))
    test_eq(result, 0)

# Test with nonexistent path
result = tags(path='/nonexistent/path')
test_eq(result, 1)

# Test with no templates dir
with tempfile.TemporaryDirectory() as tmpdir:
    old_cwd = os.getcwd()
    try:
        os.chdir(tmpdir)
        result = tags()
        test_eq(result, 1)
    finally:
        os.chdir(old_cwd)

Tags in templates:
  <Language-Name> ✓
  <Project-Current-Phase> ✓
  <Project-Name> ✓
  <Project-Objective> ✓
  <Project_Name> ✗ (not defined)
  <User-Alias> ✓
  <User-Language-Note> ✓
  <User-Profile> ✓
  <date> ✗ (not defined)
  <file_name> ✗ (not defined)
  <name> ✗ (not defined)
  <session_history> ✗ (not defined)
Tags in ../templates/Cursor/project.mdc:
  <Project-Current-Phase> ✓
  <Project-Name> ✓
  <Project-Objective> ✓
Error: Path not found: /nonexistent/path
Templates directory not found at /private/var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/tmpzjfgolkh/templates
Create 'templates/' or specify path with --path


## build

In [12]:
#| export
@call_parse
def build(
    vendor: str|None = None,  # Specific vendor to build (e.g., 'Cursor')
    dry_run: bool = False  # Show what would be done without doing it
):
    "Build templates from source"
    config = load_config()
    
    # Find base path from config location
    config_path = find_config()
    base_path = config_path.parent if config_path else Path.cwd()
    
    if dry_run:
        print("DRY RUN - No files will be created")
    
    try:
        if vendor:
            print(f"Building {vendor} templates...")
            if not dry_run:
                files = process_vendor(config, vendor, base_path)
                print(f"✓ Processed {len(files)} files")
                for f in files:
                    print(f"  {f.relative_to(base_path)}")
        else:
            print("Building all templates...")
            if not dry_run:
                results = process_all(config, base_path)
                total = sum(len(files) for files in results.values())
                print(f"✓ Processed {total} files across {len(results)} sections")
                for section, files in results.items():
                    print(f"  {section}: {len(files)} files")
        
        if not dry_run:
            print("\n✓ Build complete!")
        return 0
    except Exception as e:
        print(f"Error during build: {e}")
        return 1

In [13]:
# Test build (dry run to avoid side effects)
result = build(dry_run=True)
test_eq(result, 0)

# Test with nonexistent vendor
result = build(vendor='NonExistent')
test_eq(result, 1)

DRY RUN - No files will be created
Building all templates...
Building NonExistent templates...
Error during build: Vendor 'NonExistent' not found in config


In [14]:
#|export
@call_parse
def chelp():
    "Show help for all console scripts"
    from fastcore.xtras import console_help
    console_help('nooprompter')

In [15]:
#|exec_doc
chelp()

nooprompter               Build templates from source
nooprompter-config        Show and optionally validate configuration
nooprompter-init          Initialize nooprompter config from default template
nooprompter-tags          Discover tags in templates


----
<!-- # Colophon -->

In [55]:
#|hide
#|eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [56]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    nb_path = '04_cli.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)